<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2

In [4]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python

df_connect = pd.read_csv('data/df_connect.csv', sep=';')

DBNAME = df_connect['pasword'][0]
USER = df_connect['pasword'][1]
PASSWORD = df_connect['pasword'][2]
HOST = df_connect['pasword'][3]
PORT = df_connect['pasword'][4]

In [5]:
import warnings
warnings.filterwarnings('ignore')

connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [7]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.vacancies
'''

In [8]:
# Количество вакансий составляет 49197

df = pd.read_sql_query(query_3_1, connection)
display(df)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [65]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.employers
'''

In [66]:
# Количество работодателей составляет 23501

df = pd.read_sql_query(query_3_1, connection)
display(df)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.areas
'''

In [9]:
# Всего в выборке представлено 1362 региона

df = pd.read_sql_query(query_3_1, connection)
display(df)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.industries
'''

In [11]:
# Всего в выборке выявлено 294 сферы деятельности

df = pd.read_sql_query(query_3_1, connection)
display(df)

,count
0,294


***

In [12]:
# выводы по предварительному анализу данных

# Во всех таблицах проекта нам доступны для анализа 49197 вакансий,
# 23501 работодателей, 1362 регионоа, 294 сферы деятельности.
# Далее займемся получением более детализированной информации.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query = f'''
SELECT 
    areas.name as area,
    COUNT(vacancies.id) as cnt
FROM public.vacancies
    JOIN public.areas on vacancies.area_id = areas.id
GROUP BY areas.name
ORDER BY cnt DESC
LIMIT 5
'''

In [14]:
# Максимальное количество сосредоточено в Москве, это 5333 вакансии, что почти вдвое больше, 
# чем у ближайшего "конкурента" Санкт-Петербурга с 2851 вакансией.

df = pd.read_sql_query(query, connection)
display(df)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query = f'''
SELECT COUNT(*)
FROM public.vacancies
WHERE salary_FROM is not null or salary_to is not null
'''

In [16]:
# Около половины вакансий не имеют данных по уровню зарплаты.
# Данные по зарплате есть в 24073 вакансиях.

df = pd.read_sql_query(query, connection)
display(df)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query = f'''
SELECT 
    ROUND(AVG(salary_FROM), 0) as lower_salary,
    ROUND(AVG(salary_to), 0) as upper_salary
FROM public.vacancies
WHERE salary_FROM is not null or salary_to is not null
'''

In [18]:
# Среднее значение нижней зарплатной вилки 71065 руб,
# а верхней зарплатной вилки - 110537 руб.

df = pd.read_sql_query(query, connection)
display(df)

,lower_salary,upper_salary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query = f'''
SELECT
    crosstable.schedule,
    crosstable.employment, 
    COUNT(*) as qnt
    
-------составляем кросс-таблицу из schedule и employment-------
FROM(
    SELECT employment, schedule
    FROM (
        SELECT distinct employment
        FROM public.vacancies
        ) as employments
    CROSS JOIN
        (SELECT distinct schedule
        FROM public.vacancies
        ) as schedules
) as crosstable

---------присоединяем к кросс-таблице основную по условиям-----
    JOIN public.vacancies as vac on crosstable.employment = vac.employment
        AND crosstable.schedule = vac.schedule

GROUP BY -- группируем итоговую таблицу
    crosstable.employment, 
    crosstable.schedule

ORDER BY qnt DESC
'''

In [20]:
# Наиболее предпочитаемым сочетанием графикиа и типа трудоустройства является Полный день - Полная занятость,
# таких вакансий 35367. На втором месте Удаленная работа - Полная занятость, таких вакансий 7802.
# Остальные сочетания типов многократно уступают в популярности двум первым.

df = pd.read_sql_query(query, connection)
display(df)

,schedule,employment,qnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query = f'''
SELECT
    experience,
    COUNT(*) as qnt
FROM vacancies
GROUP BY experience
ORDER BY qnt
'''

In [22]:
# Самым востребованным является опыт от 1 до 3 лет, сотрудники с опытом более 6 лет наименее интересны работодателям.

df = pd.read_sql_query(query, connection)
display(df)

,experience,qnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [23]:
# выводы по детальному анализу вакансий
# Наибольшее количество вакансий в Москве и Санкт-Петербурге. Что предположительно коррелирует с численностью 
# населения и развитостью инфраструктуры.
# Далеко не все работодатели указывают уровень заработной платы. Предположительно это дает работодателю возможность
# находить на одну вакансию специалистов разного уровня, и устанавливать индивидуальный уровень зарплаты по результатам
# собеседования.
# В среднем уровень зарплаты выставляется работодателями от 70 до 110 тыс руб.
# Видно, что удаленный формат работы довольно востребованный, и находится на втором месте после традиционного 
# "Полный день - Полная занятость". На "удаленке" работодатели чаще всего также желают от сотрудников полной занятости.
# На удивление малая востребованность специалистов с опытом от 6 лет. Тут могут быть разные ситуации:
# - работодатель не хочет сужать круг потенциальных кандидатов
# - кто-то считает что через 5 лет в отдной специальности случается профессиональное выгорание, и лучше взять кого-то
# менее опытного, но с "огоньком в глазах".

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
query = f'''
SELECT 
    employers.name,
    qnt
FROM(
    
-----делаю таблицу из 2 строк с id работодателя-------
    (SELECT
        employer_id,
        COUNT(*) as qnt
    FROM vacancies
    GROUP BY employer_id
    ORDER BY qnt DESC
    LIMIT 1) -- выбор первого места
    
    UNION ALL -- склеиваю первую и пятую строчку
    
    (SELECT
        employer_id,
        COUNT(*) as qnt
    FROM vacancies
    GROUP BY employer_id
    ORDER BY qnt DESC
    LIMIT 1
    OFFSET 4) -- выбор пятого места
) as employers_id

----достаю наименование работодателя---------------
    JOIN employers on employers_id.employer_id = employers.id
'''

In [25]:
# На первом месте по количеству вакансий Яндекс (1933), на пятом - Газпром нефть(331).

df = pd.read_sql_query(query, connection)
display(df)

,name,qnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
query = f'''
SELECT
    area,
    employer_qnt,
    vacancies_qnt
FROM
-----создание таблицы с группировкой по работодателям и городам-----------
    (SELECT
        areas.name as area,
        areas.id,
        COUNT(distinct employers.id) as employer_qnt
    FROM areas
        JOIN employers on areas.id = employers.area
    GROUP BY areas.name, areas.id
    ) as employers_group
    
LEFT JOIN -- соединяю сгруппированные таблицы

-----создание таблицы с группировкой по вакансиям и городам-----------
        (SELECT 
            vacancies.area_id,
            COUNT(*) as vacancies_qnt
        FROM vacancies
        GROUP BY vacancies.area_id
        ) as vacancies_group
        
----------соединение полученных таблиц по городам------------        
    on employers_group.id = vacancies_group.area_id     

ORDER BY vacancies_qnt DESC nulls first, employer_qnt DESC
'''

In [27]:
# Наибольшее количество работодателей без вакансий оказалось в "регионе" Россия, что не вполне корректно с точки зрения
# определения самого региона. Такое произошло, потому что при многие работодатели позиционируют себя не в рамках
# конкретной локации, а в рамках страны. При этом в вакансиях указывают конкретный город для работы.
# Если смотреть по регионам, то локации с наибольшим количеством работодателей совпадают с локациями наибольших количеств вакансий, которые вычисляли выше.

df = pd.read_sql_query(query, connection)
display(df)

,area,employer_qnt,vacancies_qnt
0,Россия,410,NaN
1,Казахстан,207,NaN
2,Московская область,75,NaN
3,Краснодарский край,19,NaN
4,Ростовская область,18,NaN
...,...,...,...
649,"Михнево (Московская область, Ступинский район)",1,1.0
650,Электроугли,1,1.0
651,Лысьва,1,1.0
652,Тихорецк,1,1.0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query = f'''
SELECT
    employers.name,
    COUNT(distinct area_id) as area_qnt
FROM vacancies
    JOIN employers on vacancies.employer_id = employers.id
GROUP BY employers.name
ORDER BY area_qnt DESC
'''

In [29]:
# Чемпионами по охвату регионов вакансиями являются IT-компании Яндекс и Ростелеком, 181 и 152 региона соответственно.

df = pd.read_sql_query(query, connection)
display(df)

,name,area_qnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query = f'''
SELECT COUNT(*)
FROM employers
    LEFT JOIN employers_industries as e_i on employers.id = e_i.employer_id
WHERE industry_id is null
'''

In [31]:
# Оказалось 8419 работодателя не указали сферу деятельности.

df = pd.read_sql_query(query, connection)
display(df)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query = f'''
SELECT
    employers.name,
    COUNT(industry_id) as qnt
FROM employers
    LEFT JOIN employers_industries as e_i on employers.id = e_i.employer_id
GROUP BY employers.name
having COUNT(industry_id) = 4
ORDER BY 1
LIMIT 1
OFFSET 2

'''

In [33]:
# На третьем месте с четырьмя сферами деятельности оказалась компания 2ГИС

df = pd.read_sql_query(query, connection)
display(df)

,name,qnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query = f'''
SELECT
    COUNT(*)
FROM employers
    JOIN employers_industries as e_i on employers.id = e_i.employer_id
    JOIN industries on e_i.industry_id = industries.id
WHERE industries.name = 'Разработка программного обеспечения'

'''

In [35]:
# Разработка програмного обеспечения как сфера деятельности указана у 3553 работодателей.

df = pd.read_sql_query(query, connection)
display(df)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import urllib.parse as req

# Получение ссылки в нужном виде
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

df_mln = pd.read_html(url)[1]
srt_del = str('[a]')
df_mln['Город'] = df_mln['Город'].str.replace(srt_del, '')
df_mln['Город'] = df_mln['Город'].str.replace('[', '')
df_mln['Город'] = df_mln['Город'].str.replace(']', '')

# Сохраняю список городов в виде кортежа для дальнейшего использования в запросе.
city_list = tuple(df_mln['Город'])
print(city_list)


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [37]:
# текст запроса
query = f'''

---Таблица с группировкой по городам и сортировкой по количеству---
(SELECT
    areas.name,
    COUNT(*) as qnt
FROM employers
    JOIN vacancies on employers.id = vacancies.employer_id
    JOIN areas on vacancies.area_id = areas.id
WHERE employers.name = 'Яндекс'
    AND areas.name in {city_list}
GROUP BY areas.name
ORDER BY 2 )

UNION ALL --Склеивание таблицы с итогами

---Таблица с общим количеством заявок---
SELECT 'Total', sum(qnt)
FROM(
    SELECT
    areas.name,
    COUNT(*) as qnt
FROM employers
    JOIN vacancies on employers.id = vacancies.employer_id
    JOIN areas on vacancies.area_id = areas.id
WHERE employers.name = 'Яндекс'
    AND areas.name in {city_list}
GROUP BY areas.name
) as total    
'''

In [38]:
# Всего в городах - милионниках Яндекс разместил 485 вакансий, больше всего в Москве и СПб,
# менее всего в Омске и Челябинске.
df = pd.read_sql_query(query, connection)
display(df)

,name,qnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

In [39]:
# выводы по анализу работодателей
# По количеству вакансий лидируют представители IT-сектора (Яндекс, Ростелеком),
# довольно близок к ним банковский сектор (Тинькофф, Сбер), пятерку лидеров замыкает Газпром нефть.
# Локации с самыми большими скоплениями работодателей совпадают со скоплениями вакансий.
# Это города: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.
# Наибольший охват регионов вакансиями у Яндекса и Ростелекома, 181 и 152 региона соответственно.
# Яндекс оказался очень плодовит на вакансии, только по городам-миллионникам 485 вакансий. Дополнительно
# провел и сследование вакансий Яндекса во Владивостоке (код в разделе "Дополнительные исследования"). По
# Владивостоку оказалось 27 вакансий от Яндекс, но после первых трех уровни оплаты идут несоизмеримо
# низкие для сдешнего уровня жизни. Например, продукты питания и первой необходимости стоят ощутимо дороже,
# чем в Москве, проживание тоже недешевое. Думаю, что закрыть вакансии с уровнем 15-30 тыс руб нормальными кадрами во 
# Владивостоке задача не из простых. Вероятно, отсюда и такое немалое количество вакансий.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query = f'''
SELECT COUNT(*)
FROM public.vacancies
where name ilike '%data%' or name ilike '%данн%' 
'''

In [41]:
# Всего в данной выборке 1771 вакансия, связанная с данными.

df = pd.read_sql_query(query, connection)
display(df)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# Сохраняю набор условий для дальнейшего использования в запросах
condition_ds = str("""
    LOWER(name) LIKE '%data scientist%' or 
    LOWER(name) LIKE '%data science%' or
    LOWER(name) LIKE '%исследователь данных%' or
    name LIKE '%ML%' AND name  not LIKE '%HTML%' or
    LOWER(name) LIKE '%machine learning%' or
    LOWER(name) LIKE '%машинн%обучен%'""")

# текст запроса
query = f'''
SELECT COUNT(*)
FROM vacancies
WHERE 
    ({condition_ds})
    AND
    (LOWER(name) LIKE '%junior%' or
    experience = 'Нет опыта' or
    employment = 'Стажировка')   
'''

In [43]:
# для начинающего дата-сайентиста есть 51 подходящая вакансия

df = pd.read_sql_query(query, connection)
display(df)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
query = f'''
SELECT COUNT(*)
FROM vacancies
WHERE 
    ({condition_ds})
    AND
    (LOWER(key_skills) LIKE '%sql%' or LOWER(key_skills) LIKE '%postgres%' )   
'''

In [45]:
# SQL или postgres востребованы в 201 вакансии
df = pd.read_sql_query(query, connection)
display(df)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
query = f'''
SELECT COUNT(*)
FROM vacancies
WHERE 
    ({condition_ds})
    AND
    (LOWER(key_skills) LIKE '%python%')   
'''

In [47]:
# Python оказался еще более популярен, чем SQL, и указан как ключевой навык в 351 вакансии.

df = pd.read_sql_query(query, connection)
display(df)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
query = f'''
SELECT 
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR (9),'')) + 1), 2) as qnt
FROM vacancies
WHERE 
    ({condition_ds}) 
'''

In [49]:
# В среднем в вакансиях для DS указывают 6-7 ключевых навыков.

df = pd.read_sql_query(query, connection)
display(df)

,qnt
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос SELECT 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `SELECT 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
# текст запроса
query = f'''
SELECT
    experience,
    ROUND(AVG(salary),0) as salary_mean
FROM(
    (SELECT 
        experience,
        (salary_FROM + salary_to) / 2 as salary
    FROM vacancies
    WHERE 
        ({condition_ds})
        AND
        (salary_FROM is not null AND salary_to is not null)
    )  
      
    UNION ALL
    
    (SELECT 
        experience,
        coalesce(salary_FROM, 0) + coalesce(salary_to, 0) as salary
    FROM vacancies
    WHERE 
        ({condition_ds})
        AND
        (salary_FROM is not null AND salary_to is null 
            or salary_FROM is null AND salary_to is not null)
    )
) as salary_all
GROUP BY experience
'''

In [51]:
# Для соискателей в области DS без опыта работы, в среднем работодатели указывают зарплату 75 тыс руб.
# С опытом работы от 1 до 3 лет уровень ЗП поднимается до 140 тыс, а с опытом от 3 до 6 лет средний уровень ЗП 243 тыс руб.

df = pd.read_sql_query(query, connection)
display(df)

,experience,salary_mean
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [59]:
# Всего в выборке найдено более 1700 вакансий, связанных с данными. В качестве ключевых навыков python указывают чаще, 
# чем SQL, хотя SQL тоже довольно востребован. 
# Уровень зарплаты дата-сайентиста практически прямопропорционален опыту работы.


# Общий вывод по проекту

In [53]:
#     1. Одним из вопросов для размышлений после проведенных исследований для меня осталась особенность работодателя
# не указывать уровень зарплаты. Логично предположить, что, указав уровень, снимается много вопросов со стороны
# соискателей, и происходит естественная фильтрация кандидатов. Яркий пример есть в дополнительных исследованиях,
# где, условно, Яндекс говорит: "Да, у меня много вакансий во Владивостоке, уровень оплаты 15 тыс руб". И это сразу
# отсеивает соискателей, которые и не против поработать в Яндексе, но не за такую зарплату. 
# С другой стороны, если работодатель не указывает уровень зарплаты, значит, он готов выслушать предложение от 
# соискателя. И это дает возможность соискателю получить желаемый уровень ЗП, а работодателю получить сотрудника
# нужной квалификации. Данный метод, на мой взгляд, подходит для набора специалистов высокого уровня. И если
# соискатель считает себя таковым, то стоит рассматривать вакансии без указания уровня ЗП.
#     2. Из тех вакансий, где уровень зарплаты указывается, зарплатная вилка получилась около 70-110 тыс руб.
# А для дата-сайентистов вилка 74-243 тыс руб, что делает данную отрасль в два раза привлекательнее, чем 
# "среднестатистичские" вакансии.
#     3. Большое количество вакансий не обязательно у крупных и именитых компаний. Пример тому "Поляков Денис Иванович"
# у которого 127 вакансий компьютерных мастеров в 88 регионах. Подробнее в дополнительных исследованиях.

# Дополнительные исследования

В блоке про анализ работодателей заинтересовал некий работодатель "Поляков Денис Иванович", разместивший 
вакансии в 88 регионах, по охвату регионов попал в ТОП 5 наряду с Яндексом и Ростелекомом. Узнаем, сколько каких вакансий
предлагает этот работодатель, какой в среднем начальный уровень зарплаты, какой график работы.

In [54]:
# текст запроса
query = f'''
SELECT
    employers.name,
    vacancies.name,
    vacancies.schedule,
    round(AVG(salary_from), 0) as salary_from,
    COUNT(vacancies.name) as qnt_vacancies
    
FROM vacancies
    JOIN employers on  vacancies.employer_id = employers.id

WHERE vacancies.employer_id = '5130287'
GROUP BY vacancies.name, vacancies.schedule, employers.name
'''

In [55]:
df = pd.read_sql_query(query, connection)
display(df)

,name,name,schedule,salary_from,qnt_vacancies
0,Поляков Денис Иванович,Компьютерный мастер,Полный день,57568.0,74
1,Поляков Денис Иванович,Компьютерный мастер,Сменный график,60000.0,5
2,Поляков Денис Иванович,Ученик компьютерного мастера,Полный день,31458.0,48


In [56]:
# Предлагаемые вакансии это Компьютерный мастер	 и Ученик компьютерного мастера	 с начальным уровнем
# зарплаты около 30 и 60 тыс руб соответственно. Основной график работ - Полный день.

В блоке про города-милионники насчитали 485 вакансий от Яндекс. Дополнительно хочу узнать, сколько вакансий
от Яндекс есть в моем городе Владивосток, построить рейтинг вакансий по уровню зарплаты.

In [57]:
# текст запроса
query = f'''
(SELECT
    'Владивосток' as city,
    'Всего вакансий от Яндекс во Владивостоке' as vacancies_name ,
    COUNT(vacancies.name) as salary_max
FROM employers
    JOIN vacancies on employers.id = vacancies.employer_id
    JOIN areas on vacancies.area_id = areas.id
WHERE employers.name = 'Яндекс'
    AND areas.name in ('Владивосток')
)

UNION ALL 

(SELECT
    areas.name,
    vacancies.name,
    ----Выбираю максимум из зарплатной вилки-------
    CASE WHEN vacancies.salary_to is null THEN vacancies.salary_from
        ELSE vacancies.salary_to END salary_max
FROM employers
    JOIN vacancies on employers.id = vacancies.employer_id
    JOIN areas on vacancies.area_id = areas.id
WHERE employers.name = 'Яндекс'
    AND areas.name in ('Владивосток')
ORDER BY salary_max DESC NULLS LAST
)
'''

In [58]:
# Во Владивостоке оказалось 27 вакансий, подавляющее большинство это специалисты поддержки.
# Максимальный уровень зарплаты из указанных это 75000р, вакансия Разработчик (поисковая выдача).
# Это довольно невысокий уровень для Владивостока. После первых трех вакансий уровень снижается ниже
# 30 000р, прожить на такую зп во Владивостоке весьма проблематично. Но может подойти студентам без опыта в 
# качестве подработки.

df = pd.read_sql_query(query, connection)
display(df)

,city,vacancies_name,salary_max
0,Владивосток,Всего вакансий от Яндекс во Владивостоке,27.0
1,Владивосток,Разработчик (поисковая выдача),75000.0
2,Владивосток,Специалист службы поддержки в Директ,60000.0
3,Владивосток,Тимлид клиентского сервиса,45000.0
4,Владивосток,Специалист поддержки в Yandex Cloud,27000.0
5,Владивосток,Эксперт по ответам на вопросы о Яндекс Бизнесе,26000.0
6,Владивосток,Специалист поддержки в Яндекс Бизнес,26000.0
7,Владивосток,Специалист службы поддержки в Яндекс Бизнес,26000.0
8,Владивосток,Специалист службы поддержки с техническими зна...,15000.0
9,Владивосток,Специалист службы поддержки с техническими зна...,15000.0


In [6]:
# Закрываем соединение
connection.close()